# ライブラリのインポート　ファイルの読み込み

In [34]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/handson_jleague
!pip install -q japanize-matplotlib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor as RF
# データを表示する時に列を省略しないためのおまじない
pd.options.display.max_columns=None

from matplotlib import pyplot as plt
%matplotlib inline

# #日本語化matplotlibのインポート & 日本語のフォントを指定
import japanize_matplotlib
plt.rcParams['font.family'] = 'IPAexGothic'

from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error as MSE

# データの読み込み
train = pd.read_csv("./input/train_all.csv")
test = pd.read_csv("./input/test_all.csv")
sample = pd.read_csv("./input/sample_submit.csv",header=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/handson_jleague'
/content/drive/MyDrive/handson_jleague


# 特徴量エンジニアリング

### 月、日、曜日変換

In [35]:
#月、日、曜日を取り出す
train['MONTH']=train['gameday'].apply(lambda x: int(x[0:2]))
train["MONTH"] = train["MONTH"].astype(int)
train['day']=train['gameday'].apply(lambda x: int(x[3:5]))
train["WEEK"] = train["gameday"].apply(lambda x : x[x.find("(")+1:x.find("(")+2])
#テストデータも同様に
test['MONTH']=test['gameday'].apply(lambda x: int(x[:2]))
test["MONTH"] = test["MONTH"].astype(int)
test['day']=test['gameday'].apply(lambda x: int(x[3:5]))
test["WEEK"] = test["gameday"].apply(lambda x : x[x.find("(")+1:x.find("(")+2])

# 第１節などのカラムから数字を取り出してmatch_numに入れる
train['match_num']=train['match'].apply(lambda x: int(x[x.find("第")+1: x.find("節")]))
test['match_num']=test['match'].apply(lambda x: int(x[x.find("第")+1: x.find("節")]))

#不要カラムの削除
train=train.drop(columns=['gameday','match'])
test=test.drop(columns=['gameday','match'])

In [36]:
# WEEKデータの加工

train['WE_SAT'] = ((train['WEEK']== '土')).astype(int)
train['WE_SUN'] = ((train['WEEK']== "日")).astype(int)

test['WE_SAT'] = ((test['WEEK']== "土")).astype(int)
test['WE_SUN'] = ((test['WEEK']== "日")).astype(int)


# いらなそうなのでMONTH列削除
train = train.drop(columns='WEEK')
test = test.drop(columns='WEEK')

### チーム名変換

In [37]:
# ザスパ草津からザスパクサツ群馬に改名していたらしいので変更
train=train.replace('ザスパ草津','ザスパクサツ群馬')
test=test.replace('ザスパ草津','ザスパクサツ群馬')

### 天気表記の変更

In [38]:
# 天気の表記が細かすぎるものがあるので文字列を変更
def simple_weather(weather):
  if '雪' in weather:
    return '雨'
  elif '雨' in weather:
    return '雨'
  elif '曇' in weather:
    return '曇'
  elif '晴' in weather:
    return '晴'
  else:
    return weather

train['weather_simple']=train['weather'].apply(simple_weather)
test['weather_simple']=test['weather'].apply(simple_weather)

### テレビ局を番号化

In [39]:
train["tv_num"] = train["tv"].apply(lambda x : len(x.split("／")))
test["tv_num"] = test["tv"].apply(lambda x : len(x.split("／")))

## 外れ値除去と正規化

In [40]:
#外れ値（観客数０）の除去
train=train[train['y']>0]

#log1p(numpyの関数)を適用、対数をとる
train['y']=np.log1p(train["y"])

# 各会場のキャパについても正規化しておく（データのスケールを整える）
train['capa']=np.log1p(train['capa'])
test['capa']=np.log1p(test['capa'])

## ワンホットエンコーディング

In [41]:
# 制度に大きく影響のありそうなデータを選んで新しいデータフレームを作る
train_select=train[['capa','WE_SAT',"WE_SUN",'stage','MONTH','home','away','weather_simple','tv_num','id','y']]
test_select=test[['capa','WE_SAT',"WE_SUN",'stage','MONTH','home','away','weather_simple','tv_num','id']]

In [42]:
#一時的にデータを結合
# 結合前にtrainというフラグを作り、trainデータは1、testデータは0とする
train_select["train"]=1
test_select["train"]=0

# 結合
full_df = pd.concat([train_select, test_select],sort=False)

# ダミー変数化（ワンホットエンコーディング）
full_df = pd.get_dummies(full_df)

# trainフラグが1のものをtrain_selectとする。
train_select = full_df[full_df["train"]==1]
# trainフラグを削除
train_select = train_select.drop(columns="train")

# trainフラグが0のものをtest_selectとする。その後、trainフラグを削除
test_select = full_df[full_df["train"]==0]
test_select = test_select.drop(columns="train")

<ipython-input-42-20f7e769686b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_select["train"]=1
<ipython-input-42-20f7e769686b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_select["train"]=0


### 月を変換

In [43]:
# MONTHデータの加工
# J1の試合で11月の試合１２月の試合、J2の試合で11月の試合を判定するカラムの作成
train_select['J1_Nov'] = ((train_select['stage_Ｊ１']== 1)&(train_select['MONTH']==11)).astype(int)
train_select['J1_Dec'] = ((train_select['stage_Ｊ１']== 1)&(train_select['MONTH']==12)).astype(int)
train_select['J2_Nov'] = ((train_select['stage_Ｊ２']== 1)&(train_select['MONTH']==11)).astype(int)
test_select['J1_Nov'] = ((test_select['stage_Ｊ１']== 1)&(test_select['MONTH']==11)).astype(int)
test_select['J1_Dec'] = ((test_select['stage_Ｊ１']== 1)&(test_select['MONTH']==12)).astype(int)
test_select['J2_Nov'] = ((test_select['stage_Ｊ２']== 1)&(test_select['MONTH']==11)).astype(int)

# いらなそうなのでMONTH列削除
train_select = train_select.drop(columns='MONTH')
test_select = test_select.drop(columns='MONTH')

## ランキングの特長量を作成

### チームランクの作成

In [44]:
train_rank = pd.read_csv("./input/train_all2.csv")
test_rank = pd.read_csv("./input/test_all2.csv")

In [45]:
train_select = pd.merge(train_select, train_rank[['id', 'home_rank', 'away_rank']], on='id', how='left')
test_select = pd.merge(test_select, test_rank[['id', 'home_rank', 'away_rank']], on='id', how='left')


In [46]:
#一時的にデータを結合
# 結合前にtrainというフラグを作り、trainデータは1、testデータは0とする
train_select["train"]=1
test_select["train"]=0

# 結合
full_df = pd.concat([train_select, test_select],sort=False)

# ダミー変数化（ワンホットエンコーディング）
full_df = pd.get_dummies(full_df)

# trainフラグが1のものをtrain_selectとする。
train_select = full_df[full_df["train"]==1]
# trainフラグを削除
train_select = train_select.drop(columns="train")
train_select = train_select.drop(columns="id")

# trainフラグが0のものをtest_selectとする。その後、trainフラグを削除
test_select = full_df[full_df["train"]==0]
test_select = test_select.drop(columns="train")
test_select = test_select.drop(columns="id")

## データ書き出し

In [47]:
test_select.to_csv("./input/test_use_features.csv",encoding="utf8")
train_select.to_csv("./input/train_use_features.csv",encoding="utf8")

# モデル作成

## データの分割

In [ ]:
# 学習データを訓練とテストに分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_select, train['y'], random_state=0)

## 重回帰モデル
### モデルの選定をもっとやれば精度高まる？

In [ ]:
from sklearn.linear_model import LinearRegression

# LinearRegression クラスのインスタンスを作成
lr = LinearRegression()

# データに対して学習を実行
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred_train = lr.predict(X_train)
y_train_output = np.expm1(y_train)# 訓練データの目的変数yを指数化
y_pred_train_output = np.expm1(y_pred_train) # 訓練データの予測結果を指数化

# RMSEで評価
from sklearn.metrics import mean_squared_error as MSE
rmse_train = np.sqrt(MSE(y_train_output, y_pred_train_output))
print(rmse_train)

# テストデータでもRMSE評価
y_pred_test=lr.predict(X_test)
y_test_output = np.expm1(y_test)
y_pred_test_output = np.expm1(y_pred_test)

rmse_test = np.sqrt(MSE(y_test_output,y_pred_test_output))
print(rmse_test)

3133.5748931595103
3338.456484522069


## 保存

In [ ]:
# モデルの保存 model/MMDDhhmm.csvの形で保存（月日時分）
predict = lr.predict(test_select)
predict_out = np.expm1(predict)

sample[1]=predict_out
sample.to_csv('model/01202148.csv' ,header=None, index=False)

In [ ]:
# 使ったデータフレームを保存して加工を不要にする
train_select.to_csv("./input/train_select.csv",encoding="utf8")
test_select.to_csv("./input/test_select.csv",encoding="utf8")

In [ ]:
# コンペ投稿用 使用したカラムの確認
train_select.columns

Index(['capa', 'WE_SAT', 'WE_SUN', 'tv_num', 'stage_Ｊ１', 'stage_Ｊ２',
       'home_アビスパ福岡', 'home_アルビレックス新潟', 'home_カターレ富山', 'home_カマタマーレ讃岐',
       'home_ガイナーレ鳥取', 'home_ガンバ大阪', 'home_ギラヴァンツ北九州', 'home_コンサドーレ札幌',
       'home_サガン鳥栖', 'home_サンフレッチェ広島', 'home_ザスパクサツ群馬', 'home_ジェフユナイテッド千葉',
       'home_ジュビロ磐田', 'home_セレッソ大阪', 'home_ファジアーノ岡山', 'home_ベガルタ仙台',
       'home_モンテディオ山形', 'home_ロアッソ熊本', 'home_ヴァンフォーレ甲府', 'home_ヴィッセル神戸',
       'home_京都サンガF.C.', 'home_名古屋グランパス', 'home_大分トリニータ', 'home_大宮アルディージャ',
       'home_川崎フロンターレ', 'home_徳島ヴォルティス', 'home_愛媛ＦＣ', 'home_東京ヴェルディ',
       'home_松本山雅ＦＣ', 'home_柏レイソル', 'home_栃木ＳＣ', 'home_横浜Ｆ・マリノス', 'home_横浜ＦＣ',
       'home_水戸ホーリーホック', 'home_浦和レッズ', 'home_清水エスパルス', 'home_湘南ベルマーレ',
       'home_鹿島アントラーズ', 'home_ＦＣ岐阜', 'home_ＦＣ東京', 'home_ＦＣ町田ゼルビア',
       'home_Ｖ・ファーレン長崎', 'away_アビスパ福岡', 'away_アルビレックス新潟', 'away_カターレ富山',
       'away_カマタマーレ讃岐', 'away_ガイナーレ鳥取', 'away_ガンバ大阪', 'away_ギラヴァンツ北九州',
       'away_コンサドーレ札幌', 'away_サガン鳥栖', 'away_サンフレッチェ広島', 'away